In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [2]:
df = pd.read_csv("100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# tokenize
def tokenize(text):
    text = text.lower()
    text = text.replace("?", "")
    text = text.replace("'", "")
    return text.split()

In [4]:
tokenize("How are you?")

['how', 'are', 'you']

In [5]:
# Vocab
vocab = {"<UNK>": 0}

In [6]:
def build_vocab(row):
    tokenized_question = tokenize(row["question"])
    tokenized_answer = tokenize(row["answer"])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [7]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):
    indexed_text = []

    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab["<UNK>"])
    return indexed_text

In [10]:
text_to_indices("What is python?", vocab)

[1, 2, 0]

In [15]:
# load dataset
class QADataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        numerical_question = text_to_indices(
            self.df.iloc[index]["question"], self.vocab
        )
        numerical_answer = text_to_indices(self.df.iloc[index]["answer"], self.vocab)
        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = QADataset(df, vocab)

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, answer in dataloader:
    print(question, answer[0])
    break

tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])


In [19]:
# build a model
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        self.rnn = nn.RNN(50, 64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

In [20]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1, 6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [21]:
learning_rate = 0.001
epochs = 20

In [22]:
model = SimpleRNN(len(vocab))

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# training loop
for epoch in range(epochs):
    total_loss = 0
    for question, answer in dataloader:
        optimizer.zero_grad()

        # forward pass
        output = model(question)

        # loss -> output shape [1,324] - [1]
        loss = criterion(output, answer[0])

        # gradients
        loss.backward()

        # update
        optimizer.step()

        total_loss = total_loss + loss.item()
    print(f"Epoch: {epoch+1}, Loss:{total_loss:4f}")

Epoch: 1, Loss:523.613347
Epoch: 2, Loss:455.560724
Epoch: 3, Loss:378.222371
Epoch: 4, Loss:316.625904
Epoch: 5, Loss:266.327883
Epoch: 6, Loss:218.660432
Epoch: 7, Loss:174.969653
Epoch: 8, Loss:136.774130
Epoch: 9, Loss:105.013283
Epoch: 10, Loss:79.561949
Epoch: 11, Loss:61.450012
Epoch: 12, Loss:48.061654
Epoch: 13, Loss:38.005224
Epoch: 14, Loss:30.223275
Epoch: 15, Loss:25.033953
Epoch: 16, Loss:21.197112
Epoch: 17, Loss:17.360493
Epoch: 18, Loss:14.740996
Epoch: 19, Loss:12.607360
Epoch: 20, Loss:10.889609


In [27]:
def predict(model, question, threshold=0.5):
    # convert question to numbers
    numerical_question = text_to_indices(question, vocab)

    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # send to model
    output = model(question_tensor)

    # convert lofits to probs
    probs = torch.nn.functional.softmax(output, dim=1)

    # find index of max prob
    value, index = torch.max(probs, dim=1)

    if value < threshold:
        print("I don't know")

    print(list(vocab.keys())[index])

In [28]:
predict(model, "What is the largest planet in our solar system?")

jupiter
